In [1]:
from IPython.core.debugger import set_trace
from torchtext.datasets import WikiText103
import spacy
import re
import html
from torchtext import data
from spacy.symbols import ORTH
import torch
import torch.nn as nn
import torch.nn.functional as V
import torch.optim as optim

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
! ls data/

DukeMTMC-reID  DukeMTMC-reID.zip  imdb_lm  __MACOSX  market1501


In [3]:
spacy_en  = spacy.load('en')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [4]:
def tokenizer(x):
    return [tok.text for tok in spacy_en.tokenizer(x)]

In [5]:
TEXT = data.Field(lower=True, tokenize = tokenizer)


In [ ]:
train, valid, test = WikiText103.splits(TEXT)

In [ ]:
train

In [ ]:
TEXT.build_vocab(train, vectors = "fasttext.en.300d")

In [9]:
train_iter, valid_iter, test_iter = data.BPTTIterator.splits(train, valid, test,
batch_size=32,
bppt_len=30,
device=0,
repeat=False)

NameError: name 'test' is not defined

100%|█████████▉| 2518955/2519370 [05:41<00:00, 7365.51it/s]

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz,
                 dropout=0.5, tie_weights=True):
        super(LanguageModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid,ntoken)
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()
        self.hidden
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
 
    def forward(self, input):
        emb = self.drop(self.encoder(input))
        output, self.hidden = self.rnn(emb, self.hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1))
 
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return (V(weight.new(self.nlayers, bsz, self.nhid).zero_().cuda()),
                V(weight.new(self.nlayers, bsz, self.nhid).zero_()).cuda())
  
    def reset_history(self):
        self.hidden = tuple(V(v.data) for v in self.hidden)

In [ ]:
weight_matrix = TEXT.vocab.vectors
model = RNNModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 1, BATCH_SIZE)
model.encoder.weight.data.copy_(weight_matrix)
model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters, lr = 1e-3, betas=(0.7,0.99))

In [ ]:
from tqdm import tqdm
def train_epoch(epoch):
    epoch_loss = 0
    for batch in tqdm(train_iter):
        model.reset_history()
    
        optimizer.zero_grad()
        
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        loss.backward()
        
        optimizer.step()
        
        epoch_loss = loss.data[0] * prediction.size(0) * prediction.size(1)
        
        epoch_loss /= len(train.examples[0].text)
        
        val_loss = 0
        
    for batch in valid_iter:
        model.reset_history()
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        val_loss += loss.data[0] * text.size(0)
        val_loss /= len(valid.examples[0].text) 
        
        
         print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, epoch_loss, val_loss))
        